In [ ]:
#==================================================================
#Program: blockAssembly
#Version: 1.0
#Author: David Helminiak
#Date Created: September 4, 2024
#Date Last Modified: September 4, 2024
#Changelog: 1.0 - Assembly and cropping - September 2024
#Description: Assemmbles original lossless .tif block images into a WSI (using .jpg of WSI as reference),
#             then crops the WSI, so block extraction process can be matched between samples 2-70 and S70+. 
#Operation: Move back into main program directory before running.
#Status: Deprecated - As not all of the blocks from the WSI are present (excluded due to background levels), complete WSI cannot be reversed out. 
#==================================================================

#Have the notebook fill more of the display width
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:80% !important; }</style>"))

#RNG seed value to ensure run-to-run consistency (-1 to disable)
manualSeedValue = 0

#Debugging mode
debugMode = False

asciiFlag = False

#Load external libraries
exec(open("./CODE/EXTERNAL.py", encoding='utf-8').read())



In [ ]:
#When splitting WSI images, what size should the resulting blocks be (default: 400)
#Should remain consistent with block sizes given for training
blockSize = 400

#Store directory references
dir_data = '.' + os.path.sep + 'DATA' + os.path.sep
dir_blocks_data = dir_data + 'BLOCKS' + os.path.sep
dir_blocks_inputBlocks = dir_blocks_data + 'INPUT_BLOCKS' + os.path.sep
file_blocks_labels = dir_blocks_inputBlocks + 'Patch_list.xlsx'
dir_blocks_inputWSI = dir_blocks_data + 'INPUT_WSI' + os.path.sep
dir_data_reassembledWSI = dir_blocks_data + 'OUTPUTS_REASSEMBLED' + os.path.sep

#Create storage location for reassembled WSI
if os.path.exists(dir_data_reassembledWSI): shutil.rmtree(dir_data_reassembledWSI)
os.makedirs(dir_data_reassembledWSI)

#Get list of all the available blocks
metadata = pd.read_excel(file_blocks_labels, header=None, names=['name', 'label'], converters={'name':str,'label':str})
blockNamesAll_blocks, blockLabelsAll_blocks = np.array(metadata['name']), np.array(metadata['label'])
blockSampleNamesAll_blocks = np.array([re.split('PS|_', blockName)[1] for blockName in blockNamesAll_blocks])
blockFilenamesAll_blocks = [dir_blocks_inputBlocks + 'S' + blockSampleNamesAll_blocks[blockIndex] + os.path.sep + blockNamesAll_blocks[blockIndex] + '.tif' for blockIndex in range(0, len(blockNamesAll_blocks))]



In [ ]:
#Process each of the samples for which blocks exist
for sampleName in tqdm(np.unique(blockSampleNamesAll_blocks), desc='Samples', leave=True, ascii=asciiFlag):

    #Isolate blocks for the sample
    blockIndices = np.where(blockSampleNamesAll_blocks == sampleName)[0]
    blockFilenames = np.asarray(blockFilenamesAll_blocks)[blockIndices]
    
    #Load the .jpg version of the sample WSI
    imageWSI = cv2.cvtColor(cv2.imread(dir_blocks_inputWSI + sampleName + '.jpg', cv2.IMREAD_UNCHANGED), cv2.COLOR_BGR2RGB)

    #Find each block location and copy in the .tif data
    rebuilt = np.zeros((imageWSI.shape), dtype='uint8')
    mask = np.zeros((imageWSI.shape[0],imageWSI.shape[1]), dtype='uint8')
    for blockFilename in tqdm(blockFilenames, desc='Blocks', leave=False, ascii=asciiFlag):
        blockImage = cv2.cvtColor(cv2.imread(blockFilename, cv2.IMREAD_UNCHANGED), cv2.COLOR_BGR2RGB)
        heat_map = cv2.matchTemplate(imageWSI, blockImage, cv2.TM_CCOEFF_NORMED)
        y, x = np.unravel_index(np.argmax(heat_map), heat_map.shape)
        rebuilt[y:y+blockSize, x:x+blockSize] = blockImage
        mask[y:y+blockSize, x:x+blockSize] = 1
    
    #Verify all blocks were uniquely assigned
    if np.sum(mask) != len(blockFilenames)*blockSize*blockSize: 
        print('\nError - The total number of pixels in the original blocks does not match with the number that were assigned in the reassembled image.')
        break
    
    #Export the reassembled tif
    writeSuccess = cv2.imwrite(dir_data_reassembledWSI+sampleName+'_reassembled.tif', cv2.cvtColor(rebuilt, cv2.COLOR_RGB2BGR), params=(cv2.IMWRITE_TIFF_COMPRESSION, 1))
    
